In [1]:
import os
import sys
print("Python version: ",sys.version)
print("Version info:   ",sys.version_info)

# add path to get to the modules renderneuron and parallelframe
sys.path.insert(0,'../src/')
import neuronswc as ns
import neuronmeshgenerator as nmg

Python version:  3.8.10 (default, Mar 13 2023, 10:26:41) 
[GCC 9.4.0]
Version info:    sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)
JSON:            2.0.9
networkx:        3.1
scipy:           1.10.1


In [3]:
neuron_filename='../cells/0-2a.CNG.swc'
num_ref=8
start_dx=32.0
n_cir_points=12
sph_contours=10
sph_points=16
MESHFOLDER=nmg.make_meshes(neuron_filename,start_dx,num_ref,n_cir_points,sph_contours,sph_points)
nmg.write_vrn(MESHFOLDER)

folder not found
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement0.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement0.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment0.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement1.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement1.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment1.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement2.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement2.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment2.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement3.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement3.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment3.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement4.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement4.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment4.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement5.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement5.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment5.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement6.swc